# Kafka Streams и KSQLDB

## Подготовка

### Создать топик

Топик будет отслеживать местоположения грузовиков:

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --create \
    --topic trucks-locations \
    --if-not-exists \
    --partitions 4 \
    --replication-factor 1

### Создать схему

Схема содержит координаты грузовика:

In [ ]:
cat <<EOF | sed 's,",\\",g;1s,^,{"schema" : ",;$s,$,"},' > /tmp/trucks-locations-schema.json
{
   "fields" : [
      {
         "name" : "id",
         "type" : "string"
      },
      {
         "name" : "latitude",
         "type" : "double"
      },
      {
         "name" : "longitude",
         "type" : "double"
      }
   ],
   "name" : "TruckLocationEvent",
   "namespace" : "com.github.neshkeev.kafka.avro.locaitons",
   "type" : "record"
}
EOF

In [ ]:
curl -s http://schema-registry:8081/subjects/trucks-locations-value/versions \
    -X POST \
    -H 'Content-Type: application/vnd.schemaregistry.v1+json' \
    -d '@/tmp/trucks-locations-schema.json' | json_pp

In [ ]:
curl -s http://schema-registry:8081/subjects | json_pp

Функция `ksql-execute` абстрагирует логику выполнения `ksql` запросов:

In [ ]:
function ksql-execute() {
    local script=${1}

    ksql --query-timeout 20000 \
        --file "${script}" \
        -- http://${KSQL_SERVER_HOST}:${KSQL_SERVER_PORT}
}

## KSQL

**ksqlDB** - это база данных поверх Apache Kafka, которая позволяет создавать стриминговые приложения. Основными абстракциями ksqlDB являются:

- Kafka stream (стрим) - абстракция над потенциально неограниченным потоком данных;
- Kafka materialized view/table (материализованное представление/таблица) - абстракция текущего состояния.

**Kafka Stream** позволяет установить связь между схемой данных и топиком, следовательно, можно быть уверенным, что все сообщения, записанные в топик через стрим, будут соответствовать схеме данных.

При создании стрима, создаются так же схема и топик, если их еще нет, причем формат названия схемы будет следующим: `ИМЯ_СТРИМА`-`value`, а название топика указывается в параметрах стрима.

Kafka materialized view позволяет отслеживать последние изменения в потоке данных и актуализировать текущее состояние.

### Создание стрима

При создании стрима **ksqlDB** автоматически создаст схему и топик для стрима при необходимости:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/create-users-stream.cli <<EOF
CREATE STREAM users (
    id VARCHAR,
    name INT
) WITH (
    kafka_topic='users',
    value_format='avro',
    partitions=2
);
EOF

In [ ]:
ksql-execute /tmp/create-users-stream.cli

Появилась новая схема `users-value`:

In [ ]:
curl -s http://schema-registry:8081/subjects | json_pp

Создался новый топик `users`:

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --describe \
    --topic users

Инструкция `CREATE STREAM` позволяет задать лишь небольшой набор конфигов нового топика (`partitions`, `replicas`), а поэтому дополнительную конфигурацию топика необходимо выполнять через `kafka-configs`:

In [ ]:
kafka-configs --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --entity-type topics \
    --entity-name users \
    --alter \
    --add-config 'retention.ms=3600000'

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --describe \
    --topic users

### Создание стрима по схеме

При наличии схемы можно указать ее идентификатор в конфигах стрима и исключить список колонок. ksqlDB самостоятельно определит колонки и их типы на базе указанной схемы.

Для координат грузовиков актуальная версия схемы:

In [ ]:
curl -s http://schema-registry:8081/subjects/trucks-locations-value/versions

Актуальная версия будет передана в качестве параметра для `value_schema_id` при создании стрима:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/create-trucks-locations-stream.cli <<EOF
CREATE STREAM trucks_locations WITH (
    kafka_topic='trucks-locations',
    value_format='AVRO',
    value_schema_id=1,
    partitions=4
);
EOF

In [ ]:
ksql-execute /tmp/create-trucks-locations-stream.cli

Если посмотреть детали стрима, то можно увидеть какие колонки были созданы по схеме:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/describe-trucks-stream.cli <<EOF
DESCRIBE trucks_locations;
EOF

In [ ]:
ksql-execute /tmp/describe-trucks-stream.cli

При этом можно отметить, что новая схема не создалась:

In [ ]:
curl -s http://schema-registry:8081/subjects | json_pp

Новая версия схемы `trucks-locations-value` не появилась:

In [ ]:
curl -s http://schema-registry:8081/subjects/trucks-locations-value/versions

### Регистрозависимость

Инструкции **ksqlDB** так же как и в реляционных базах данных являются регистронезависимыми, а значит можно выполнять запросы не обращая внимания на регистр. Но главной особенностью создания стрима на базе существующей схемы является тот факт, что колонки стрима становятся регистрозависимыми:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/select-trucks-stream-bad.cli <<EOF
SELECT id, latitude, longitude
  FROM trucks_locations;
EOF

In [ ]:
ksql-execute /tmp/select-trucks-stream-bad.cli

Необходимо взять имена колонок в квычки:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/select-trucks-stream-good.cli <<EOF
SELECT "id", "latitude", "longitude"
  FROM trucks_locations;
EOF

In [ ]:
ksql-execute /tmp/select-trucks-stream-good.cli

### Таблицы

Стрим `trucks_locations` представляет собой поток событий об изменении местоположения грузовиков. Для того, чтобы получить текущее положение грузовика, необходимо создать таблицу на базе стрима:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/create-current-truck-locations-table.cli <<EOF
CREATE TABLE current_trucks_locations AS
  SELECT "id" as id,
         LATEST_BY_OFFSET("latitude") AS la,
         LATEST_BY_OFFSET("longitude") AS lo
  FROM trucks_locations
  GROUP BY "id"
  EMIT CHANGES;
EOF

In [ ]:
ksql-execute /tmp/create-current-truck-locations-table.cli

Пояснения:
- [`LATEST_BY_OFFSET`](https://docs.ksqldb.io/en/latest/developer-guide/ksqldb-reference/aggregate-functions/#latest_by_offset) - это функция аггрегации, которая выбирает последнее событие по оффсету в топике;
- [`EMIT CHANGES`](https://docs.ksqldb.io/en/latest/developer-guide/ksqldb-reference/select-push-query/#emit) говорит о том, что данные в таблице будут автоматически обновляться по мере поступления новых событий в топик.

На базе таблиц можно создавать другие таблицы:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/create-trucks-near-moscow-table.cli <<EOF
CREATE OR REPLACE TABLE trucks_near_moscow AS
  SELECT ROUND(GEO_DISTANCE(la, lo, 55.75, 37.50), -1) AS distance,
         COLLECT_LIST(id) AS trucks,
         COUNT(*) AS count
  FROM current_trucks_locations
  GROUP BY ROUND(GEO_DISTANCE(la, lo, 55.75, 37.50), -1);
EOF

In [ ]:
ksql-execute /tmp/create-trucks-near-moscow-table.cli

### Запись событий

Запись событий в стрим можно выполнить при помощи `INSERT` инструкции:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/insert-trucks-locations.cli <<EOF
INSERT INTO trucks_locations ("id", "latitude", "longitude") VALUES ('c2309eec', 55.75, 37.51);
INSERT INTO trucks_locations ("id", "latitude", "longitude") VALUES ('18f4ea86', 55.751, 37.507);
INSERT INTO trucks_locations ("id", "latitude", "longitude") VALUES ('4ab5cbad', 55.767, 37.60);
INSERT INTO trucks_locations ("id", "latitude", "longitude") VALUES ('8b6eae59', 55.1, 37.38);
INSERT INTO trucks_locations ("id", "latitude", "longitude") VALUES ('4a7c7b41', 56.15, 38);
INSERT INTO trucks_locations ("id", "latitude", "longitude") VALUES ('4ddad000', 53.85, 37.89);
EOF

In [ ]:
ksql-execute /tmp/insert-trucks-locations.cli

### Pull Запросы

Pull запросы позволяют получить текущее состояние. Для каждого ключа вычисляется самое последнее состояние, и после этого запрос завершается. Этим они напоминают обычные `select` запросы в реляционных базах данных:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/get-trucks-10-km-moscow.cli <<EOF
SELECT *
  FROM trucks_near_moscow
 WHERE distance <= 10;
EOF

In [ ]:
ksql-execute /tmp/get-trucks-10-km-moscow.cli

### Push Запросы

Push запросы позволяют в реальном времени получать сообщения об изменениях, а значит push запросы никогда не завершаются самостоятельно. Остановить запрос можно одним из двух вариантов:
- остановить запрос принудительно (`CTRL + c`),
- установить таймаут на выполнение запроса.

Выбрать грузовики, которые находятся в 15 километрах от Москвы:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/get-trucks-15-km-moscow.cli <<EOF
SELECT *
  FROM trucks_locations
 WHERE GEO_DISTANCE("latitude", "longitude", 55.75, 37.50) <= 15
  EMIT CHANGES;
EOF

Таймаут на выполнение запроса через консольную утилиту `ksql` устанавливается через параметр `--query-timeout`. Например, следующий запрос завершится через 20 секунд:

In [ ]:
ksql --query-timeout 20000 \
    --file /tmp/get-trucks-15-km-moscow.cli \
    -- http://${KSQL_SERVER_HOST}:${KSQL_SERVER_PORT}

### Задание

1. Загрузить схему `products` в Schema Registry через REST API:
```bash
cat <<EOF > /tmp/product-value.json
{
    "schema": "{ \
        \"type\": \"record\", \
        \"name\": \"Product\", \
        \"namespace\": \"com.github.neshkeev.kafka.avro\", \
        \"fields\":[ \
            { \
                \"name\":\"id\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"name\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"price\", \
                \"type\":\"double\" \
            } \
        ] \
    }" \
}
EOF
```
2. Подтвердить наличие схемы `products` любым из доступных вариантов;
2. Создать стрим `products` и топик `products` на базе схемы `products`;
2. Установить конфигурацию для топика `products`:
    - `partitions=4`;
    - `replication-factor=1`;
    - [`retention.ms=1800000`](https://docs.confluent.io/platform/current/installation/configuration/topic-configs.html#retention-ms);
    - [`compression-type=gzip`](https://docs.confluent.io/platform/current/installation/configuration/topic-configs.html#compression-type).
2. Подтвердить установку конфигураций для топика;
2. Вставить записи в стрим `products`:
```sql
INSERT INTO products ("id", "name", "price") VALUES ('$RANDOM', 'фен', 1337.51);
INSERT INTO products ("id", "name", "price") VALUES ('$RANDOM', 'микроволновая печь', 7432.15);
INSERT INTO products ("id", "name", "price") VALUES ('$RANDOM', 'стиральная машина', 23461.99);
INSERT INTO products ("id", "name", "price") VALUES ('$RANDOM', 'фен', 999.99);
INSERT INTO products ("id", "name", "price") VALUES ('$RANDOM', 'микроволновая печь', 5941.87);
```
7. Получить актуальные цены по каждому товару;
7. Создать таблицу с актуальными ценами по каждому товару, данные в которой постоянно обновляются.

## Kafka Streams

**Kafka Streams** - это клиентская библиотека для обработки стримов и трансформации данных в топиках из java коде. Источниками и приениками данных являются топики в Apache Kafka кластере. При помощи Kafka Streams можно достичь тех же результатов, которые доступны в **ksqlDB**.

Идеи Kafka Streams будет рассмотрены на примере приложения для прослушивания музыки:

- есть набор песен,
- пользователи могут прослушивать песни целиком или частично,
- если длительность прослушивания песни была меньше 20 секунд, то считается, что воспроизведение песни было активировано случайно (пользователь не намеревался послушать эту песню).

### Подготовка

#### Регистрация схем

Перед продолжением необходимо зарегистировать схемы для стримов:
- песни,
- воспроизведение песен.

Схемы находятся в java проекте:

In [ ]:
execute \
ls kafka-java-demo/src/main/avro

Для регистрации схем можно использовать [`schema-registry-maven-plugin`](https://docs.confluent.io/platform/current/schema-registry/develop/maven-plugin.html#schema-registry-register):

In [ ]:
mvn schema-registry:register > /tmp/register-schemas.log;
    tail -n 10 /tmp/register-schemas.log

Проверка схем в Schema Registry:

In [ ]:
curl -s http://schema-registry:8081/subjects |
    json_pp |
    grep '\[\|play\|song\|\]'

#### Создание стримов

Для генерации данных необходимо создать стримы:

1. Функция `extract-schema-id` позволяет найти идентификатор схемы по имени схемы:

In [ ]:
function extract-schema-id() {
    local schema_name=${1}

    curl -s http://schema-registry:8081/subjects/${schema_name}/versions/latest |
        json_pp |
        sed -n '/"id"/s,.* \([0-9]\+\)\,,\1,p'
}

2. Подготовка скриптов для создания стримов:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/create-songs-streams.cli <<EOF
CREATE STREAM songs WITH (
    kafka_topic='songs',
    value_format='AVRO',
    value_schema_id=$(extract-schema-id song-value),
    partitions=4
);

CREATE STREAM songs_count WITH (
    kafka_topic='songs-count',
    value_format='AVRO',
    value_schema_id=$(extract-schema-id song-count-value),
    partitions=4
);

CREATE STREAM play_events WITH (
    kafka_topic='play-events',
    value_format='AVRO',
    value_schema_id=$(extract-schema-id play-event-value),
    partitions=4
);

CREATE STREAM play_events_actual WITH (
    kafka_topic='play-events-actual',
    value_format='AVRO',
    value_schema_id=$(extract-schema-id play-event-value),
    partitions=4
);

CREATE STREAM play_events_accidental WITH (
    kafka_topic='play-events-accidental',
    value_format='AVRO',
    value_schema_id=$(extract-schema-id play-event-value),
    partitions=4
);


DESCRIBE songs;
DESCRIBE songs_count;
DESCRIBE play_events;
DESCRIBE play_events_actual;
DESCRIBE play_events_accidental;
EOF

3. Исполнение скриптов для создания стримов:

In [ ]:
ksql-execute /tmp/create-songs-streams.cli

#### Генерация данных

Записать данные о песнях:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/insert-songs.cli <<EOF
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (1,'Fresh Fruit For Rotting Vegetables','Dead Kennedys','Chemical Warfare','Punk');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (2,'We Are the League','Anti-Nowhere League','Animal','Punk');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (3,'Live In A Dive','Subhumans','All Gone Dead','Punk');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (4,'PSI','Wheres The Pope?','Fear Of God','Punk');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (5,'Totally Exploited','The Exploited','Punks Not Dead','Punk');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (6,'The Audacity Of Hype','Jello Biafra And The Guantanamo School Of Medicine','Three Strikes','Punk');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (7,'Licensed to Ill','The Beastie Boys','Fight For Your Right','Hip Hop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (8,'De La Soul Is Dead','De La Soul','Oodles Of Os','Hip Hop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (9,'Straight Outta Compton','N.W.A','Gangsta Gangsta','Hip Hop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (10,'Fear Of A Black Planet','Public Enemy','911 Is A Joke','Hip Hop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (11,'Curtain Call - The Hits','Eminem','Fack','Hip Hop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (12,'21','Adele','Rolling in the Deep','Pop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (13,'In The Lonely Hour','Sam Smith','Stay With Me','Pop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (14,'The Calling','Hilltop Hoods','The Calling','Hip Hop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (15,'x','Ed Sheeran','Thinking Out Loud','Pop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (16,'V','Maroon 5','Sugar','Pop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (17,'This Is What The Truth Feels Like','Gwen Stefani','Red Flag','Pop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (18,'This Is Acting','Sia','Alive','Pop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (19,'24K Magic','Bruno Mars','That is What I Like','Pop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (20,'Black Sunday','Cypress Hill','Insane in the Brain','Hip Hop');
INSERT INTO songs("id", "name", "artist", "album", "genre") VALUES (21,'Aquemini','Outkast','Aquemini','Hip Hop');
EOF

In [ ]:
ksql-execute /tmp/insert-songs.cli

Сгенерация событий с проигрыванием песен:

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/insert-play-events.cli <<<$(
for i in {1..100}; do
    printf 'INSERT INTO play_events("id", "duration") VALUES (%d, %d);\n' $((RANDOM % 21)) $((RANDOM % 100))
done
)

In [ ]:
ksql-execute /tmp/insert-play-events.cli

### Основная задача

Необходимо определить, является ли событие о воспроизведении песни намеренным или случайным. Случайное воспроизведение относится к прослушиваниям, которые заняли меньше 20 секунд.

### Наивный подход к трансформации данных

Часто возникает необходимость преобразовать уже имеющиеся данные в топике и залить результат в другой топик. Очевидным выглядит алгоритм:
1. прочитать данные из топика,
1. выполнить преобразование данных,
1. записать преобразованный результат в новый топик.

Именно такой подход реализован в `com.github.neshkeev.kafka.streams.NaiveTransformations`:

In [ ]:
docker compose cp /kafka-exercises/kafka-java-demo/src/main/java/com/github/neshkeev/kafka/streams/NaiveTransformations.java manager:/home/jovyan/work

[NaiveTransformations.java](NaiveTransformations.java)

In [ ]:
mvn clean compile \
    exec:java -Dexec.mainClass='com.github.neshkeev.kafka.streams.NaiveTransformations' \
        > /tmp/naive-transformation.log ;
    tail /tmp/naive-transformation.log

Указанный подход хотя и решает поставленную задачу, но сильно ограничен в применении: если в будущем нужна будет более сложная трансформация или во взаимодействии будет участвовать больше топиков, то модификация такого класса будет сильно затруднена.

Также можно заметить, что в классе очень много служебного кода, который занимается подготовкой консьюмера и продюссера, а реальный алгоритм умещается лишь в одном тернарном операторе, который выбирает один или другой топик для записи.

### Трансформация данных через Kafka Streams

**Kafka Streams** - это клиентская библиотека, которая содержит множество стандартных алгоритмов (`map`, `filter` и т.д.), что значительно облегчает код и позволяет программисту сфокусироваться только на бизнес-логике.

Приложение на базе Kafka Streams состоит из трёх частей:

- топология ([`org.apache.kafka.streams.Topology`](https://kafka.apache.org/23/javadoc/org/apache/kafka/streams/Topology.html)) - указывает какие трансформации участвуют в работе приложения. Представляется в виде графа;
- конфигурация ([`org.apache.kafka.streams.StreamsConfig.*`](https://kafka.apache.org/23/javadoc/org/apache/kafka/streams/StreamsConfig.html)) - конфигурация приложения (`bootstrap.servers` и т.д.) аналогична конфигурации для `Producer` и `Consumer`;
- клиент ([`org.apache.kafka.streams.KafkaStreams`](https://kafka.apache.org/23/javadoc/org/apache/kafka/streams/KafkaStreams.html)) - клиентский класс, который позволяет запускать трансформации.

В свою очередь топология строится на двух концепциях:

- стримы ([`org.apache.kafka.streams.kstream.KStream`](https://kafka.apache.org/23/javadoc/org/apache/kafka/streams/kstream/KStream.html)) - абстракция над потенциально неограниченным потоком данных
- таблицы - абстракция над текущим состоянием. Таблицы бывают двух типов:
    - неизменяемая таблица ([`org.apache.kafka.streams.kstream.GlobalKTable`](https://kafka.apache.org/23/javadoc/org/apache/kafka/streams/kstream/GlobalKTable.html)) - применяется для хранения данных, которые не меняются в процессе жизни приложения. Можно использовать, например, для справочной информации;
    - таблица с текущим состоянием ([`org.apache.kafka.streams.kstream.KTable`](https://kafka.apache.org/23/javadoc/org/apache/kafka/streams/kstream/KTable.html)) - применяется для отслеживания текущего состояния по ключу.

**Kafka Streams** позволяет реализовать логику аналогичную **ksqlDB** запросам на уровне java кода и встроить ее в java приложение.

In [ ]:
docker compose cp /kafka-exercises/kafka-java-demo/src/main/java/com/github/neshkeev/kafka/streams/ActualAccidentalPlayEvents.java manager:/home/jovyan/work

[ActualAccidentalPlayEvents.java](ActualAccidentalPlayEvents.java)

In [ ]:
mvn clean compile \
    exec:java \
        -Dexec.mainClass='com.github.neshkeev.kafka.streams.ActualAccidentalPlayEvents' \
        -Dexec.args='30' \
        > /tmp/streams-transformation.log ;
    tail /tmp/streams-transformation.log

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/play-events-actual.cli <<EOF
SELECT sum("duration") as total, "id" as id
  FROM play_events_actual
 GROUP BY "id"
  EMIT CHANGES;
EOF

In [ ]:
ksql-execute /tmp/play-events-actual.cli

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/play-events-accidental.cli <<EOF
SELECT sum("duration") as total, "id" as id
  FROM play_events_accidental
 GROUP BY "id"
  EMIT CHANGES;
EOF

In [ ]:
ksql-execute /tmp/play-events-accidental.cli

### Соединение (join)

In [ ]:
docker compose cp /kafka-exercises/kafka-java-demo/src/main/java/com/github/neshkeev/kafka/streams/GetSongsPlayCount.java manager:/home/jovyan/work

[GetSongsPlayCount.java](GetSongsPlayCount.java)

In [ ]:
mvn clean compile \
    exec:java \
        -Dexec.mainClass='com.github.neshkeev.kafka.streams.GetSongsPlayCount' \
        -Dexec.args='30' \
        > /tmp/songs-count-streams.log ;
    cat /tmp/songs-count-streams.log

In [ ]:
HOST=${KSQL_CLI_HOST} \
new_file /tmp/songs-count.cli <<EOF
SELECT *
  FROM songs_count;
EOF

In [ ]:
ksql-execute /tmp/songs-count.cli

### Задание

При помощи **kafka streams**:

1. сделать join между двумя стримами на базе топиков `song` и `play-events`;
1. найти какие композиции чаще всего включали случайно (топик `play-events-accidental`);
1. получить песни с одинаковой продолжительностью из топике `play-events-actual`;
1. на базе строковых столбцов в топике `songs` реализовать алгоритм [`word count`](https://en.wikipedia.org/wiki/Word_count): посчитать сколько раз каждое слово встречается в тексте;
1. реализовать предыдущие задачи при помощи запросов **ksqlDB**.